In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
import os
import kagglehub
import shutil

# Download the dataset
try:
  path = kagglehub.dataset_download("sanikamal/horses-or-humans-dataset")

  shutil.move(path, '/content')
except:
  print("error")

100%|██████████| 307M/307M [00:02<00:00, 117MB/s]

Extracting files...


error


In [ ]:
from PIL import Image

img = Image.open('/content/1/horse-or-human/train/horses/horse01-0.png')
img.show()

In [ ]:
print(f"Image format: {img.format}")
print(f"Image size: {img.size}")
print(f"Image mode: {img.mode}")

Image format: PNG
Image size: (300, 300)
Image mode: RGBA


In [ ]:
path = '/content/1/horse-or-human/'
path_train = path+'train'
path_valid = path+'validation'
### data is extracted already and its not labelled so we use the tensorflow imagedata generator
## to label the images

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
 rescale=1./255,
 rotation_range=40,
 width_shift_range=0.2,
 height_shift_range=0.2,
 shear_range=0.2,
 zoom_range=0.2,
 horizontal_flip=True,
 fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
 path_train,
 target_size=(300, 300),
 class_mode='binary'
)

Found 1027 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([
 tf.keras.layers.Conv2D(16, (3,3), activation='relu' ,
 input_shape=(300, 300, 3)),
 tf.keras.layers.MaxPooling2D(2, 2),
 tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
 tf.keras.layers.MaxPooling2D(2,2),
 tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
 tf.keras.layers.MaxPooling2D(2,2),
 tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
 tf.keras.layers.MaxPooling2D(2,2),
 tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
 tf.keras.layers.MaxPooling2D(2,2),
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation='relu'),
 tf.keras.layers.Dense(1, activation='sigmoid')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(
    loss='binary_crossentropy',
    optimizer=RMSprop(0.001),
    metrics=['accuracy']
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = train_datagen.flow_from_directory(
 path_valid,
 target_size=(300, 300),
 class_mode='binary'
)

Found 256 images belonging to 2 classes.


In [ ]:
### adding the call back for the more than 99 percent accuracy

In [ ]:
import tensorflow as tf

class EarlyStoppingAtAccuracy(tf.keras.callbacks.Callback):
    def __init__(self, accuracy_threshold=0.9):
        super(EarlyStoppingAtAccuracy, self).__init__()
        self.accuracy_threshold = accuracy_threshold

    def on_epoch_end(self, epoch, logs=None):
        accuracy = logs.get("accuracy")  # Tr300aining accuracy
        if accuracy is not None and accuracy >= self.accuracy_threshold:
            print(f"\nReached {self.accuracy_threshold * 100:.2f}% accuracy, stopping training!")
            self.model.stop_training = True

# Create an instance of the callback
early_stopping_callback = EarlyStoppingAtAccuracy(accuracy_threshold=0.99)

# Train the model with the callback
history = model.fit(
    train_generator,
    epochs=20,  # Set a higher number of epochs; training will stop early when accuracy reaches 99%.
    validation_data=validation_generator,
    callbacks=[early_stopping_callback]
)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


33/33 ━━━━━━━━━━━━━━━━━━━━ 312s 9s/step - accuracy: 0.5317 - loss: 0.7138 - val_accuracy: 0.5039 - val_loss: 0.6850
Epoch 2/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 311s 9s/step - accuracy: 0.6139 - loss: 0.6578 - val_accuracy: 0.5000 - val_loss: 0.8968
Epoch 3/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 294s 8s/step - accuracy: 0.6830 - loss: 0.5754 - val_accuracy: 0.5078 - val_loss: 0.8073
Epoch 4/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 313s 9s/step - accuracy: 0.7599 - loss: 0.4867 - val_accuracy: 0.5117 - val_loss: 1.7897
Epoch 5/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 296s 8s/step - accuracy: 0.8267 - loss: 0.4403 - val_accuracy: 0.5234 - val_loss: 1.6044
Epoch 6/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 314s 9s/step - accuracy: 0.8621 - loss: 0.3367 - val_accuracy: 0.5430 - val_loss: 2.7872
Epoch 7/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 306s 9s/step - accuracy: 0.8564 - loss: 0.3523 - val_accuracy: 0.5586 - val_loss: 1.1724
Epoch 8/20
33/33 ━━━━━━━━━━━━━━━━━━━━ 304s 9s/step - accuracy: 0.9077 - loss: 0.2611 - val_accuracy: 0.5820 - val_loss: 2.

In [ ]:
### using the model for predictions

from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  path = "/content/"+fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  image_tensor = np.vstack([x])
  classes = model.predict(image_tensor)

  if classes[0]>0.5:
    print("human")
  else:
    print("horse")

Saving horse01-1.png to horse01-1.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 805ms/step
horse


In [ ]:
!mkdir 'model'

model.save('model/hh.keras')

In [1]:
from huggingface_hub import login
login(token="")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


LocalTokenNotFoundError: Token is required (`token=True`), but no token found. You need to provide a token or be logged in to Hugging Face with `huggingface-cli login` or `huggingface_hub.login`. See https://huggingface.co/settings/tokens.

In [ ]:
### upload this model into the hugging face or kaggle

from tensorflow.keras.models import save_model
from huggingface_hub import HfApi, HfFolder

# Upload the model manually
from huggingface_hub import HfApi

api = HfApi()

try:
  repo_url = api.create_repo(repo_id="navidfalah/Horse_Human_classification_tf", private=False)
except:
  pass

api.upload_folder(
    folder_path="/content/model",
    repo_id="navidfalah/Horse_Human_classification_tf"
)